In [1]:
#Script for finding correct parameter values that gives back the Higgs mass at zero temperature.
#Begin by defining an effective potential w/o radiative corrections. 

import numpy as np
import cmath
import matplotlib.pyplot as plt
from scipy.misc import derivative
from scipy import optimize

In [2]:
#yukawa coupling constant of fermion f 
y_f = 0.99                                          #(for the top quark)

#EW - sector coupling constants g (SU(2)_L) and g_p (U(1)_Y) 
g = 0.65
g_p = 0.36
eps = 0.00001

# Effective mass of fermion f acquired through yukawa coupling w/ the Higgs
def m_f(phi):
    return y_f**2/2*phi**2 + eps

# W and Z boson masses
def m_W(phi):
    return (g**2/4)*phi**2 + eps

def m_Z(phi):
    return (g**2 + g_p**2)*phi**2/4 + eps

# Higgs and Goldstone masses
def m_h(phi, mu, lam, Lam):                                               
    return - 3*lam*phi**2 - mu + (15/4)*phi**4/Lam**2 + eps

#m_h 0 at h_min = (2/5)*(lam/c6)*(1 - np.sqrt(1-5/3*c6*(mu**2)/lam**2))

def m_g(phi, mu, lam, Lam):
    return -lam*phi**2 - mu + (3/4)*phi**4/Lam**2 + eps

In [3]:
# different parts of the 1loop effective potential calculated in the ms-scheme. For the first order correction,
# separating the contribution coming from gauge bosons and other particles. obs! input for the first order corrections are 
# the squared masses, i.e. m**2 and not m.  

def v_tree(phi, mu, lam, Lam):
    return -mu/2*phi**2 - lam/4*phi**4 + phi**6/(8*Lam**2)

def v_loop_g(m):
    scale = 246
    return m**2/(64*np.pi**2)*(cmath.log(m/scale) - 5/6)

def v_loop(m):
    scale = 246
    return m**2/(64*np.pi**2)*(cmath.log(m/scale) - 3/2)


def v_1(phi, mu, lam, Lam):
    return np.real(v_loop(m_h(phi, mu, lam, Lam)) + 3*v_loop(m_g(phi, mu, lam, Lam)) - 12*v_loop(m_f(phi))
             + 3*v_loop_g(m_Z(phi)) + 6*v_loop_g(m_W(phi)))

def v_eff(phi, mu, lam, Lam):
    return np.real(v_tree(phi, mu, lam, Lam) + v_1(phi, mu, lam, Lam))

In [4]:
# defining the minimum and second derivative (at the minimum) of the 1-loop potential as a function of the parameter values 
# 
def phi_min(mu,lam,Lam):
    #initial guess to where the minimum can be is the tree level minimum
    if mu < 0:
        phi_0 = np.sqrt((2*Lam**2)/3*(lam + np.sqrt(lam**2 - 3*mu/Lam**2)))
    else:
        phi_0 = np.sqrt((2*Lam**2)/3*(lam + np.sqrt(lam**2 + 3*mu/Lam**2)))
        
    return optimize.fmin(v_eff, phi_0, args=(mu,lam,Lam),  disp=0)[0]

def d2_v(mu,lam,Lam):
    
    phi = phi_min(mu,lam,Lam)
    return derivative(v_eff, phi, args = (mu,lam,Lam), n=2)

# function we want to minimimize wrt to parameters mu, lam, Lam to give correct effective potential
def opt(x):
    mu, lam, Lam = x
    v = 246
    mh = 125
    return (phi_min(mu,lam,Lam) - v)**2 + (np.sqrt(d2_v(mu,lam,Lam)) - mh)**2

#Tree level parameters as functions of physical observables to be used as starting guesses for the optimization
def mu0(Lam):
    mh = 125
    v = 246
    return mh**2/2 - 3/4*v**4/Lam**2

def lam0(Lam):
    mh = 125
    v = 246
    return 3/2*v**2/Lam**2 - mh**2/(2*v**2)                                   

In [10]:
# Making the list of all parameters which satisfy the T=0 conditions of the effective potential.
# Starting with making list of initial conditions

Lam = np.linspace(600, 1000, 15)

x0 = []

for L in Lam:
    x0.append([mu0(L), lam0(L), L])


parameters = []

for row in x0:
    results = optimize.minimize(opt, row, tol=1e-6)
    parameters.append(results.x)

print(parameters)

[array([1.81111891e+02, 5.22413319e-02, 6.34448147e+02]), array([8.60513993e+02, 3.97386648e-02, 6.34632788e+02]), array([1.45213398e+03, 1.02135735e-02, 6.57142848e+02]), array([ 1.97186001e+03, -1.54084482e-03,  7.14295700e+02]), array([ 2.42908801e+03, -1.53618082e-02,  7.40377457e+02]), array([ 2.83697187e+03, -2.39592112e-02,  7.50381924e+02]), array([ 3.19520727e+03, -3.59667075e-02,  7.84055025e+02]), array([ 3.52143276e+03, -4.57733695e-02,  8.01357639e+02]), array([ 3.81176651e+03, -5.60412700e-02,  8.38404224e+02]), array([ 4.07490302e+03, -6.52079847e-02,  8.65610702e+02]), array([ 4.31049523e+03, -7.23587662e-02,  8.89363560e+02]), array([ 4.52822969e+03, -7.89349577e-02,  9.12920628e+02]), array([ 4.72559005e+03, -8.49665936e-02,  9.36497323e+02]), array([ 4.90429760e+03, -9.15621459e-02,  9.71754309e+02]), array([ 5.07008217e+03, -9.54399950e-02,  9.81950870e+02])]


In [11]:
# Sort the paramaeter values by the third entry i.e along Lam in ascending order
# If spacing between different Lam values is big enough, the list will probalby 
# already be sorted and this step is unneccesary.

parameters.sort(key=lambda row: (row[2]), reverse=False)
print(parameters)

[array([1.81111891e+02, 5.22413319e-02, 6.34448147e+02]), array([8.60513993e+02, 3.97386648e-02, 6.34632788e+02]), array([1.45213398e+03, 1.02135735e-02, 6.57142848e+02]), array([ 1.97186001e+03, -1.54084482e-03,  7.14295700e+02]), array([ 2.42908801e+03, -1.53618082e-02,  7.40377457e+02]), array([ 2.83697187e+03, -2.39592112e-02,  7.50381924e+02]), array([ 3.19520727e+03, -3.59667075e-02,  7.84055025e+02]), array([ 3.52143276e+03, -4.57733695e-02,  8.01357639e+02]), array([ 3.81176651e+03, -5.60412700e-02,  8.38404224e+02]), array([ 4.07490302e+03, -6.52079847e-02,  8.65610702e+02]), array([ 4.31049523e+03, -7.23587662e-02,  8.89363560e+02]), array([ 4.52822969e+03, -7.89349577e-02,  9.12920628e+02]), array([ 4.72559005e+03, -8.49665936e-02,  9.36497323e+02]), array([ 4.90429760e+03, -9.15621459e-02,  9.71754309e+02]), array([ 5.07008217e+03, -9.54399950e-02,  9.81950870e+02])]


In [12]:
# Saving the list results to be later used in other scripts 
# The values will be saved in a long list, use reshape to get the correct array of tuples with the three parameters
parameters_file = open('parameters.txt', 'w')

for row in parameters:
    np.savetxt(parameters_file, row)
    
parameters_file.close()

og_array = np.loadtxt('parameters.txt').reshape(-1,3)
print(og_array)

[[ 1.81111891e+02  5.22413319e-02  6.34448147e+02]
 [ 8.60513993e+02  3.97386648e-02  6.34632788e+02]
 [ 1.45213398e+03  1.02135735e-02  6.57142848e+02]
 [ 1.97186001e+03 -1.54084482e-03  7.14295700e+02]
 [ 2.42908801e+03 -1.53618082e-02  7.40377457e+02]
 [ 2.83697187e+03 -2.39592112e-02  7.50381924e+02]
 [ 3.19520727e+03 -3.59667075e-02  7.84055025e+02]
 [ 3.52143276e+03 -4.57733695e-02  8.01357639e+02]
 [ 3.81176651e+03 -5.60412700e-02  8.38404224e+02]
 [ 4.07490302e+03 -6.52079847e-02  8.65610702e+02]
 [ 4.31049523e+03 -7.23587662e-02  8.89363560e+02]
 [ 4.52822969e+03 -7.89349577e-02  9.12920628e+02]
 [ 4.72559005e+03 -8.49665936e-02  9.36497323e+02]
 [ 4.90429760e+03 -9.15621459e-02  9.71754309e+02]
 [ 5.07008217e+03 -9.54399950e-02  9.81950870e+02]]
